#### 현재 feature들:
- user_id
    - 특정 유저들 그룹핑 방법
- isbn
    - 베스트셀러 찾기?
- book_title
    - embedding? how?
- book_author
    - 특정 작가를 좋아하는 사람들이 있나?
- year_of_publication
    - 고전적인 책을 좋아하는 사람이 있나?
- publisher
    - 이거는 나중에 보자 의미있는지 모르겟음
- language
    - 언어별?
- category 
    - high category: 카테고리별로 다른지
        - fiction / non_fiction feature가 유의미할지
        - fiction / non_fiction으로 하는게 모든 카테고리를 고려하는 것보다 경향성이 더 좋다면, https://huggingface.co/BritishLibraryLabs/bl-books-genre?text=Poems+on+various+subjects.+Whereto+is+prefixed+a+short+essay+on+the+structure+of+English+verse 이거 사용해보는 것도 좋을 듯
- age
    - 연령대별 ~~?
- summary
    - 요약을 가지고 카테고리 결측치를 채울 수 있을까?
- location_city
    - 지역별로 rating?
- location_state
    - 내 생각에는 city만 알면 될 것 같음
- location_country
    - 내 생각에는 city만 알면 될 것 같음
    
#### target:
- ratings

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
path='./data/'

users = pd.read_csv(path+'users.csv')
books = pd.read_csv(path+'books.csv')
ratings = pd.read_csv(path+'train_ratings.csv')
test_ratings = pd.read_csv(path+'test_ratings.csv')

print('users shape: ', users.shape)
print('books shape: ', books.shape)
print('ratings shape: ', ratings.shape)

users shape:  (68092, 3)
books shape:  (149570, 10)
ratings shape:  (306795, 3)


# users 파일
결측치:
- location
    - city, state, country
        -> 같은 city, state보고 결측치 채우면 됨

In [3]:
# age에 결측치가 많다.
users.isna().sum()/len(users)

user_id     0.000000
location    0.000000
age         0.408756
dtype: float64

In [4]:
users['location'] = users['location'].str.replace(r'[^0-9a-zA-Z:,]', '') # 특수문자 제거

users['location_city'] = users['location'].apply(lambda x: x.split(',')[0].strip())
users['location_state'] = users['location'].apply(lambda x: x.split(',')[1].strip())
users['location_country'] = users['location'].apply(lambda x: x.split(',')[2].strip())

users = users.replace('na', np.nan) #특수문자 제거로 n/a가 na로 바뀌게 되었습니다. 따라서 이를 컴퓨터가 인식할 수 있는 결측값으로 변환합니다.
users = users.replace('', np.nan) # 일부 경우 , , ,으로 입력된 경우가 있었으므로 이런 경우에도 결측값으로 변환합니다.

/tmp/ipykernel_89262/3844664685.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  users['location'] = users['location'].str.replace(r'[^0-9a-zA-Z:,]', '') # 특수문자 제거


In [5]:
users.isna().sum()

user_id                 0
location                0
age                 27833
location_city         122
location_state       3254
location_country     2124
dtype: int64

In [6]:
modify_location = users[(users['location_country'].isna())&(users['location_city'].notnull())]['location_city'].values
location = users[(users['location'].str.contains('seattle'))&(users['location_country'].notnull())]['location'].value_counts().index[0]

location_list = []
for location in modify_location:
    try:
        right_location = users[(users['location'].str.contains(location))&(users['location_country'].notnull())]['location'].value_counts().index[0]
        location_list.append(right_location)
    except:
        pass

In [7]:
for location in location_list:
    users.loc[users[users['location_city']==location.split(',')[0]].index,'location_state'] = location.split(',')[1]
    users.loc[users[users['location_city']==location.split(',')[0]].index,'location_country'] = location.split(',')[2]

In [8]:
users.isna().sum()

user_id                 0
location                0
age                 27833
location_city         122
location_state       1132
location_country      271
dtype: int64

In [9]:
users[['location_city','location_state','location_country']].sample(5)

,location_city,location_state,location_country
64678,london,england,unitedkingdom
1012,NaN,kansas,usa
45101,phoenix,arizona,usa
51984,newyork,newyork,usa
54094,wildwood,missouri,usa


In [10]:
# city랑 state가 같은 것이 있던 것 같은데 왜 그런거고, 이게 맞는건지 파악해보자
display(users[users['location_city']==users['location_state']][['location_city','location_state','location_country']].sample(5))
print(len(users[users['location_city']==users['location_state']][['location_city','location_state','location_country']]))
print(users[users['location_city']==users['location_state']][['location_city','location_state','location_country']].nunique())

,location_city,location_state,location_country
14377,newyork,newyork,usa
61519,newyork,newyork,usa
45751,hamburg,hamburg,germany
58671,newyork,newyork,usa
30629,berlin,berlin,germany


2321
location_city       226
location_state      226
location_country     76
dtype: int64


In [11]:
df = users[users['location_city']=='berlin']
display(df[df['location_city'] != df['location_state']])

,user_id,location,age,location_city,location_state,location_country


In [12]:
# city랑 country가 같은 것이 있던 것 같은데 왜 그런거고, 이게 맞는건지 파악해보자
display(users[users['location_city']==users['location_country']][['location_city','location_state','location_country']].sample(5))
print(len(users[users['location_city']==users['location_country']][['location_city','location_state','location_country']]))
print(users[users['location_city']==users['location_country']][['location_city','location_state','location_country']].nunique())

,location_city,location_state,location_country
42469,hongkong,na,hongkong
58975,italy,lazio,italy
59922,singapore,na,singapore
5417,hongkong,na,hongkong
7556,hongkong,na,hongkong


174
location_city       22
location_state      25
location_country    22
dtype: int64


In [13]:
# city랑 country랑 state까지 동일한게 있는지 파악해보자
df = users[users['location_city']==users['location_country']]
display(df[users['location_city']==users['location_state']][['location_city','location_state','location_country']].sample(5))

/tmp/ipykernel_89262/3150871034.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  display(df[users['location_city']==users['location_state']][['location_city','location_state','location_country']].sample(5))


,location_city,location_state,location_country
12482,quit,quit,quit
64995,monaco,monaco,monaco
37014,panama,panama,panama
51855,brunei,brunei,brunei
1361,kuwait,kuwait,kuwait


In [14]:
display(users[users['location_city']=='unknown'])
display(users[users['location_state']=='unknown'])
display(users[users['location_country']=='unknown'])
print(users[users['location_city']=='unknown'][['location_city','location_state','location_country']].nunique())
print(users[users['location_state']=='unknown'][['location_city','location_state','location_country']].nunique())
print(users[users['location_country']=='unknown'][['location_city','location_state','location_country']].nunique())

,user_id,location,age,location_city,location_state,location_country
7584,127980,"unknown,england,unitedkingdom",NaN,unknown,england,unitedkingdom
26152,237293,"unknown,unknown,unknown",NaN,unknown,unknown,unknown
27855,214851,"unknown,unknown,usa",NaN,unknown,unknown,usa
38015,48421,"unknown,massachusetts,usa",NaN,unknown,massachusetts,usa


,user_id,location,age,location_city,location_state,location_country
26152,237293,"unknown,unknown,unknown",NaN,unknown,unknown,unknown
27855,214851,"unknown,unknown,usa",NaN,unknown,unknown,usa
56581,262270,"weisbadenaaf,unknown,germany",NaN,weisbadenaaf,unknown,germany


,user_id,location,age,location_city,location_state,location_country
26152,237293,"unknown,unknown,unknown",NaN,unknown,unknown,unknown


location_city       1
location_state      3
location_country    3
dtype: int64
location_city       2
location_state      1
location_country    3
dtype: int64
location_city       1
location_state      1
location_country    1
dtype: int64


## 해보기
2. 'unknown' 이라는 결측치 존재함 -> np.nan 으로 대체  
3. location_country == 'c' 제거  
4. country만 결측치인 것은 같은 방식으로 해결하기  
5. state만 결측치인 것도 같은 방식으로 해결하기  
6. city만 결측치인 것도 같은 방식으로 해결하기  
7. city: 각 state별 최빈 city로 채우고, state: 각 country별 최빈 state로 채우기
    - city -> state 순서로 업데이트
8. country -> city와 state로 채운 결측치로 country 채우기
9. 위에 다 처리한 거를 processed_users.csv로 저장하기 -> 이거는 계속 업데이트해야함  

In [15]:
# 2번 'unknown' 이라는 결측치 존재함 -> np.nan 으로 대체
users = users.replace('unknown', np.nan)

In [16]:
# unknown -> na 처리 완료
display(users[users['location_city']=='unknown'])
display(users[users['location_state']=='unknown'])
display(users[users['location_country']=='unknown'])
print(users[users['location_city']=='unknown'][['location_city','location_state','location_country']].nunique())
print(users[users['location_state']=='unknown'][['location_city','location_state','location_country']].nunique())
print(users[users['location_country']=='unknown'][['location_city','location_state','location_country']].nunique())

,user_id,location,age,location_city,location_state,location_country


,user_id,location,age,location_city,location_state,location_country


,user_id,location,age,location_city,location_state,location_country


location_city       0
location_state      0
location_country    0
dtype: int64
location_city       0
location_state      0
location_country    0
dtype: int64
location_city       0
location_state      0
location_country    0
dtype: int64


In [17]:
# 3번 location_country == 'c' 제거
# country 한글자 짜리 찾기
df = users.loc[~users['location_country'].isna(), 'location_country'].apply(lambda x: len(x))
df1 = list(df[df == 1].index)
df2 = list(df[df == 2].index)
df3 = list(df[df == 3].index)
print(len(df1))
print(len(df2))
print(len(df3))


1
22
45207


In [18]:
users.loc[df2]

,user_id,location,age,location_city,location_state,location_country
394,231237,"waianae,hawaii,us",NaN,waianae,hawaii,us
3553,192292,"4729pepperwoodave,longbeach,ca,usa",NaN,4729pepperwoodave,longbeach,ca
4579,111444,"washington,dc,dc,usa",NaN,washington,dc,dc
8735,244891,"washington,dc,dc,usa",NaN,washington,dc,dc
10726,266613,"washington,dc,dc,usa",NaN,washington,dc,dc
13980,49180,"330eurekacanyon,corralitos,ca,usa",NaN,330eurekacanyon,corralitos,ca
17202,154824,"washington,dc,dc,usa",32.0,washington,dc,dc
19814,121368,"1110lynnwoodblvd,nashville,tn,usa",48.0,1110lynnwoodblvd,nashville,tn
22132,187961,"aylesbury,buckinghamshire,uk,unitedkingdom",NaN,aylesbury,buckinghamshire,uk
25759,125989,"washington,dc,dc,usa",NaN,washington,dc,dc


In [19]:
users.loc[users['location_country'] == 'us', 'location_country'] = 'usa'
users.loc[users['location_country'] == 'dc', 'location_country'] = 'usa'

In [20]:
users.loc[df2]

,user_id,location,age,location_city,location_state,location_country
394,231237,"waianae,hawaii,us",NaN,waianae,hawaii,usa
3553,192292,"4729pepperwoodave,longbeach,ca,usa",NaN,4729pepperwoodave,longbeach,ca
4579,111444,"washington,dc,dc,usa",NaN,washington,dc,usa
8735,244891,"washington,dc,dc,usa",NaN,washington,dc,usa
10726,266613,"washington,dc,dc,usa",NaN,washington,dc,usa
13980,49180,"330eurekacanyon,corralitos,ca,usa",NaN,330eurekacanyon,corralitos,ca
17202,154824,"washington,dc,dc,usa",32.0,washington,dc,usa
19814,121368,"1110lynnwoodblvd,nashville,tn,usa",48.0,1110lynnwoodblvd,nashville,tn
22132,187961,"aylesbury,buckinghamshire,uk,unitedkingdom",NaN,aylesbury,buckinghamshire,uk
25759,125989,"washington,dc,dc,usa",NaN,washington,dc,usa


In [21]:
# 3번 location_country == 'c' 제거
# country 한글자 짜리 찾기
df = users.loc[~users['location_country'].isna(), 'location_country'].apply(lambda x: len(x))
df1 = list(df[df == 1].index)
df2 = list(df[df == 2].index)
df3 = list(df[df == 3].index)
print(len(df1))
print(len(df2))
print(len(df3))
# 3번 해버리기 길이 1인애랑 길이 2인애 drop



1
7
45222


In [22]:
users.loc[users['location_country'] == 'unitedkingdom']

,user_id,location,age,location_city,location_state,location_country
70,213915,"sheffield,syorkshire,unitedkingdom",29.0,sheffield,syorkshire,unitedkingdom
176,81088,"romsey,hampshire,unitedkingdom",18.0,romsey,hampshire,unitedkingdom
381,218121,"brighton,colorado,usa",NaN,brighton,england,unitedkingdom
464,4781,"manchester,newhampshire,usa",47.0,manchester,england,unitedkingdom
472,12863,"london,england,unitedkingdom",29.0,london,england,unitedkingdom
...,...,...,...,...,...,...
67984,270787,"cheddar,england,unitedkingdom",54.0,cheddar,england,unitedkingdom
68030,274057,"plymouth,england,unitedkingdom",54.0,plymouth,england,unitedkingdom
68033,274373,"hebdenbridge,england,unitedkingdom",39.0,hebdenbridge,england,unitedkingdom
68067,276927,"croydon,england,unitedkingdom",26.0,croydon,england,unitedkingdom


In [23]:
users.loc[users['location_country'] == 'england']

,user_id,location,age,location_city,location_state,location_country
762,14387,"coalville,leicestershire,england,unitedkingdom",35.0,coalville,leicestershire,england
940,38464,"tharston,norwich,england,unitedkingdom",33.0,tharston,norwich,england
1965,196146,"silsden,keighley,england,unitedkingdom",38.0,silsden,keighley,england
4580,112709,"stockport,cheshire,england",42.0,stockport,cheshire,england
6673,274004,"sydenham,london,england,unitedkingdom",41.0,sydenham,london,england
13534,10067,"watton,norfolk,england",61.0,watton,norfolk,england
16394,264346,"northrode,cheshire,england,unitedkingdom",43.0,northrode,cheshire,england
18135,97972,"dorking,surrey,england,unitedkingdom",21.0,dorking,surrey,england
20725,64316,"ermington,sdevon,england,unitedkingdom",21.0,ermington,sdevon,england
21817,1830,"kidderminster,worcestershire,england,unitedkin...",54.0,kidderminster,worcestershire,england


In [24]:
users.loc[users['location_country'] == 'scotland']

,user_id,location,age,location_city,location_state,location_country
52971,180272,"saltcoats,ayrshire,scotland",NaN,saltcoats,ayrshire,scotland


In [25]:
users.loc[users['location_country'] == 'whales']

,user_id,location,age,location_city,location_state,location_country


In [26]:
users.loc[users['location_country'] == 'northernireland']

,user_id,location,age,location_city,location_state,location_country


In [27]:
users.loc[users['location_country'] == 'scotland', 'location_country'] = 'unitedkingdom'
users.loc[users['location_country'] == 'england', 'location_country'] = 'unitedkingdom'
users.loc[users['location_country'] == 'uk', 'location_country'] = 'unitedkingdom'

In [28]:
display(users.loc[users['location_country'] == 'england'])
display(users.loc[users['location_country'] == 'scotland'])
display(users.loc[users['location_country'] == 'uk'])

,user_id,location,age,location_city,location_state,location_country


,user_id,location,age,location_city,location_state,location_country


,user_id,location,age,location_city,location_state,location_country


In [29]:
# 3번 location_country == 'c' 제거
# country 한글자 짜리 찾기
df = users.loc[~users['location_country'].isna(), 'location_country'].apply(lambda x: len(x))
df1 = list(df[df == 1].index)
df2 = list(df[df == 2].index)
df3 = list(df[df == 3].index)
print(len(df1))
print(len(df2))
print(len(df3))
# 3번 해버리기 길이 1인애랑 길이 2인애 drop

display(users.loc[df1])
display(users.loc[df2])

1
4
45222


,user_id,location,age,location_city,location_state,location_country
45233,41195,"c,c,c",NaN,c,c,c


,user_id,location,age,location_city,location_state,location_country
3553,192292,"4729pepperwoodave,longbeach,ca,usa",NaN,4729pepperwoodave,longbeach,ca
13980,49180,"330eurekacanyon,corralitos,ca,usa",NaN,330eurekacanyon,corralitos,ca
19814,121368,"1110lynnwoodblvd,nashville,tn,usa",48.0,1110lynnwoodblvd,nashville,tn
67584,278215,"nelson,motueka,nz,newzealand",28.0,nelson,motueka,nz


In [31]:
users.drop(df1, inplace=True)
users.drop(df2, inplace=True)

# 3번 location_country == 'c' 제거
# country 한글자 짜리 찾기
df = users.loc[~users['location_country'].isna(), 'location_country'].apply(lambda x: len(x))
df1 = list(df[df == 1].index)
df2 = list(df[df == 2].index)
df3 = list(df[df == 3].index)
print(len(df1))
print(len(df2))
print(len(df3))

0
0
45222


#### country는 결측치이면서 state가 결측치가 아닌 경우로 country 채우기

In [33]:
# 4. country만 결측치인 것은 같은 방식으로 해결하기
# 5. state만 결측치인 것도 같은 방식으로 해결하기
# 6. city만 결측치인 것도 같은 방식으로 해결하기
# state가 있는데 country가 결측치값인 경우는 state를 보고 채울 수 있음.

# 수정해야할 location들
# country는 결측치이면서 state가 결측치가 아닌 경우로 country 채우기
modify_location = users[(users['location_country'].isna())&(users['location_state'].notnull())]['location_state'].values

location_list = []
for location in modify_location:
    try:
        right_location = users[(users['location'].str.contains(location))&(users['location_country'].notnull())]['location'].value_counts().index[0]
        location_list.append(right_location)
    except:
        pass

In [34]:
for location in location_list:
    users.loc[users[users['location_state']==location.split(',')[1]].index,'location_country'] = location.split(',')[2]

In [35]:
users.isna().sum()

user_id                 0
location                0
age                 27830
location_city         126
location_state       1135
location_country      196
dtype: int64

#### state는 결측치이면서 city와 country가 결측치가 아닌 경우로 state로 채우기

In [37]:
# 4. country만 결측치인 것은 같은 방식으로 해결하기
# 5. state만 결측치인 것도 같은 방식으로 해결하기
# 6. city만 결측치인 것도 같은 방식으로 해결하기
# state가 있는데 country가 결측치값인 경우는 state를 보고 채울 수 있음.

# 수정해야할 location들
# state는 결측치이면서 city와 country가 결측치가 아닌 경우로 state로 채우기
modify_location = users[(users['location_state'].isna())&(users['location_city'].notnull())&(users['location_country'].notnull())]['location_city'].values

location_list = []
for location in modify_location:
    try:
        right_location = users[(users['location'].str.contains(location))&(users['location_state'].notnull())]['location'].value_counts().index[0]
        location_list.append(right_location)
    except:
        pass

In [38]:
for location in location_list:
    users.loc[users[users['location_city']==location.split(',')[0]].index,'location_state'] = location.split(',')[1]

In [39]:
users.isna().sum()

user_id                 0
location                0
age                 27830
location_city         126
location_state        610
location_country      196
dtype: int64

In [47]:
# 7. city: 각 state별 최빈 city로 채우고, state: 각 country별 최빈 state로 채우기
#     - city -> state 순서로 업데이트
# 8. country -> city와 state로 채운 결측치로 country 채우기
#유저당 몇 권 정도의 책을 읽었는지 살펴보겠습니다.
users.groupby('location_state')['location_city'].agg(pd.Series.mode).to_frame()

,location_city
location_state,
,fortyfort
322dzkie,"[321d378, 321d380]"
aargau,"[aarau, baden]"
aarhus,rhusv
ab,"[sebabeach, winfield]"
...,...
zrich,zrich
zuerich,zuerich
zug,zug


In [70]:
# state에 숫자 있는 거 확인하기
def hasNumber(stringVal):
    try:
        return any(elem.isdigit() for elem in stringVal)
    except:
        return np.nan
df1 = users['location_city'].apply(lambda x: hasNumber(x))
df2 = users['location_state'].apply(lambda x: hasNumber(x))
df3 = users['location_country'].apply(lambda x: hasNumber(x))

In [71]:
has_number_idx = list(set(list(df1[df1 == True].index) + list(df2[df2 == True].index) + list(df3[df3 == True].index)))

In [72]:
print(has_number_idx)
len(has_number_idx)

[36992, 65920, 21891, 37766, 56198, 44683, 62607, 49681, 63252, 66558, 49303, 53783, 54552, 56728, 53659, 24604, 56729, 56730, 66975, 35878, 44714, 32427, 65578, 41005, 51118, 5807, 7594, 1073, 57266, 49843, 9909, 44216, 2491, 44475, 62141, 57279, 22722, 26818, 34756, 60999, 38217, 30282, 51914, 65737, 67789, 49486, 61649, 57297, 37588, 68053, 63830, 27351, 50138, 23134, 864, 353, 49376, 8932, 65254, 66153, 47722, 33900, 2541, 24304, 13168, 53746, 57076, 58357, 42231, 38008, 58105, 55165, 54526, 6911]


74

In [73]:
display(users.loc[list(df1[df1 == True].index)])
display(users.loc[list(df2[df2 == True].index)])
display(users.loc[list(df3[df3 == True].index)])

,user_id,location,age,location_city,location_state,location_country
353,200234,"rr1battersea,ontario,canada",53.0,rr1battersea,ontario,canada
864,28800,"03770elverger,alacant,spain",69.0,03770elverger,alacant,spain
1073,57515,"8125915vilamoura,algarve,portugal",44.0,8125915vilamoura,algarve,portugal
2491,268009,"dublin12,na,ireland",41.0,dublin12,NaN,ireland
2541,276113,"321d378,322dzkie,poland",21.0,321d378,322dzkie,poland
...,...,...,...,...,...,...
66153,179594,"3014bern,na,switzerland",55.0,3014bern,NaN,switzerland
66558,193663,"23564lbeck,schleswigholstein,germany",38.0,23564lbeck,schleswigholstein,germany
66975,211984,"19258boizenburg,hamburg,germany",NaN,19258boizenburg,hamburg,germany
67789,256636,"89075ulm,badenwuerttemberg,germany",51.0,89075ulm,badenwuerttemberg,germany


,user_id,location,age,location_city,location_state,location_country
2541,276113,"321d378,322dzkie,poland",21.0,321d378,322dzkie,poland
5807,264646,"atwater,ca95301,california,usa",58.0,atwater,ca95301,california
7594,163349,"hammonton,nj08037,newjersey,usa",50.0,hammonton,nj08037,newjersey
13168,209238,"passchristian,mississippi39571,usa",60.0,passchristian,mississippi39571,usa
21891,7418,"multiple,travelingnursedo3monthsatatimeallover...",34.0,multiple,travelingnursedo3monthsatatimealloverus,usa
44683,272993,"7793burnetroad,no24,austin,usa",62.0,7793burnetroad,no24,austin
56198,205060,"efland,nc27243,northcarolina,usa",29.0,efland,nc27243,northcarolina
56728,33776,"d378,woj322dzkie,poland",32.0,d378,woj322dzkie,poland
56729,192877,"321d380,322dzkie,poland",18.0,321d380,322dzkie,poland
56730,268506,"pabianice,woj322dzkie,poland",17.0,pabianice,woj322dzkie,poland


,user_id,location,age,location_city,location_state,location_country


In [78]:
# 74개를 일일히 처리하기도 귀찮고, 이게 지금 user가 정보 입력할 때 실수한 것 같음
# 그냥 숫자 있는 city, state는 nan 처리하는게 날듯
# 숫자 처리는 country city state 처리하기 전에 먼저 해줘야함

# state에 숫자 있는 거 확인하기
def change_num2nan(stringVal):
    try:
        if any(elem.isdigit() for elem in stringVal):
            return np.nan
        else:
            return stringVal
    except:
        # np.nan인 경우
        return np.nan

df1 = users['location_city'].apply(lambda x: change_num2nan(x))
# users['location_state'] = users['location_state'].apply(lambda x: change_num2nan(x))
# users['location_country'] = users['location_country'].apply(lambda x: change_num2nan(x))

In [79]:
df1.to_frame()

,location_city
0,timmins
1,ottawa
2,NaN
3,toronto
4,victoria
...,...
68087,danville
68088,victoria
68089,irvington
68090,vancouver


In [80]:
users['location_city'] = users['location_city'].apply(lambda x: change_num2nan(x))
users['location_state'] = users['location_state'].apply(lambda x: change_num2nan(x))
users['location_country'] = users['location_country'].apply(lambda x: change_num2nan(x))

In [81]:
users.isna().sum()

user_id                 0
location                0
age                 27830
location_city         193
location_state        621
location_country      196
dtype: int64

#### 숫자인 것들 np.nan으로 바꿨기 때문에 다시 결측치 채우는 과정 적용해야함

In [82]:
modify_location = users[(users['location_country'].isna())&(users['location_city'].notnull())]['location_city'].values

location_list = []
for location in modify_location:
    try:
        right_location = users[(users['location'].str.contains(location))&(users['location_country'].notnull())]['location'].value_counts().index[0]
        location_list.append(right_location)
    except:
        pass
for location in location_list:
    users.loc[users[users['location_city']==location.split(',')[0]].index,'location_state'] = location.split(',')[1]
    users.loc[users[users['location_city']==location.split(',')[0]].index,'location_country'] = location.split(',')[2]
    
    
modify_location = users[(users['location_country'].isna())&(users['location_state'].notnull())]['location_state'].value

location_list = []
for location in modify_location:
    try:
        right_location = users[(users['location'].str.contains(location))&(users['location_country'].notnull())]['location'].value_counts().index[0]
        location_list.append(right_location)
    except:
        pass
for location in location_list:
    users.loc[users[users['location_state']==location.split(',')[1]].index,'location_country'] = location.split(',')[2]
    
    
modify_location = users[(users['location_state'].isna())&(users['location_city'].notnull())&(users['location_country'].notnull())]['location_city'].values

location_list = []
for location in modify_location:
    try:
        right_location = users[(users['location'].str.contains(location))&(users['location_state'].notnull())]['location'].value_counts().index[0]
        location_list.append(right_location)
    except:
        pass
for location in location_list:
    users.loc[users[users['location_city']==location.split(',')[0]].index,'location_state'] = location.split(',')[1]

In [83]:
users.isna().sum()

user_id                 0
location                0
age                 27830
location_city         193
location_state        617
location_country      195
dtype: int64

In [84]:
# 7. city: 각 state별 최빈 city로 채우고, state: 각 country별 최빈 state로 채우기
#     - city -> state 순서로 업데이트
# 8. country -> city와 state로 채운 결측치로 country 채우기

users.groupby('location_state')['location_city'].agg(pd.Series.mode).to_frame()

,location_city
location_state,
,fortyfort
aargau,"[aarau, baden]"
aarhus,rhusv
ab,"[sebabeach, winfield]"
aberdeenshire,"[aboyne, huntly]"
...,...
zrich,zrich
zuerich,zuerich
zug,zug


#### zrich   zurich	zurich	-> 이런 이상한 경우는 눈으로 보는 수밖에

일단은   
7. city: 각 state별 최빈 city로 채우고, state: 각 country별 최빈 state로 채우기  
     - city -> state 순서로 업데이트하고  
     - 다시 업데이트된걸로 한번더 하기  
8. country -> city와 state로 채운 결측치로 country 채우기

In [108]:
state_mode_city = users.groupby('location_state')['location_city'].agg(pd.Series.mode)
country_mode_state = users.groupby('location_country')['location_state'].agg(pd.Series.mode)

In [147]:
country_mode_state

location_country
                 nordrheinwestfalen
aberdeenshire                    uk
alabama                          al
albania                       lazio
alberta                          ab
                        ...        
westindies                   tobago
westyorkshire              keighley
wisconsin              westvirginia
worcester                        ma
yugoslavia                   serbia
Name: location_state, Length: 243, dtype: object

In [109]:
country_mode_state['aberdeenshire']

'uk'

In [110]:
city_nan_idx = users[(users['location_city'].isna())].index
state_nan_idx = users[(users['location_state'].isna())].index

In [111]:
print(users['location_city'].nunique())
print(users['location_state'].nunique())
# 최빈값으로 하기 위해서는 
# unique값이 적은 state부터 결측치 처리하고 그 결측치 기반으로 city를 처리해야함
# city처럼 unique값이 크면, state별 최빈값이 작기 때문에 state부터 처리하는게 좋음 그리고
# city부터 처리하고 state를 처리하면, city와 state가 엇갈리는 상황이 발생하기 때문임

11922
1229


In [112]:
city_nan_idx

Int64Index([    2,   353,   864,  1008,  1012,  1065,  1073,  1200,  1590,
             1679,
            ...
            66558, 66975, 67061, 67067, 67104, 67402, 67789, 67805, 68053,
            68084],
           dtype='int64', length=193)

In [113]:
state_nan_idx

Int64Index([    2,   633,   919,  1291,  1334,  1442,  1447,  1679,  1739,
             1774,
            ...
            67183, 67279, 67314, 67402, 67530, 67854, 67889, 67973, 68007,
            68075],
           dtype='int64', length=617)

In [131]:
users.sample(2)

,user_id,location,age,location_city,location_state,location_country
40809,205436,"olympia,washington,usa",25.0,olympia,washington,usa
4213,27947,"lima,ohio,usa",28.0,lima,ohio,usa


In [132]:
country_mode_state.sample(2)

location_country
ferrara    emiliaromagna
estonia               []
Name: location_state, dtype: object

In [137]:
# state_nan_idx = users[(users['location_state'].isna())].index
for idx in state_nan_idx:
    try:
        print(users.iloc[idx, 4])
        tmp = country_mode_state[users.iloc[idx, 5]]
        if isinstance(tmp, str):
            # print(country_mode_state[users.iloc[idx]['location_country']])
            users.iloc[idx, 4] = tmp
            print(users.iloc[idx, 4])
        else:
            print(5)
    except:
        continue

nan
nan
5
metromanila
metromanila
metromanila
metromanila
indiana
indiana
tokyo
tokyo
nan
5
nan
riftvalley
riftvalley
kualalumpur
kualalumpur
england
england
nan
iledefrance
iledefrance
nan
israel
israel
cairo
cairo
pennsylvania
pennsylvania
uusimaa
uusimaa
indiana
indiana
nan
karelia
karelia
california
california
nan
california
california
nan
iledefrance
iledefrance
california
california
california
california
ontario
ontario
california
california
california
california
california
california
california
california
ontario
ontario
espaa
espaa
california
california
ontario
ontario
california
california
california
california
california
california
california
california
california
california
california
california
california
california
california
california
california
california
california
california
california
california
victoria
victoria
nan
california
california
vaud
vaud
california
california
california
california
california
california
california
california
california
california
california

In [138]:
for idx in city_nan_idx:
    try:
        print(users.iloc[idx, 3])
        tmp = country_mode_state[users.iloc[idx, 4]]
        if isinstance(tmp, str):
            # print(country_mode_state[users.iloc[idx]['location_country']])
            users.iloc[idx, 3] = tmp
            print(users.iloc[idx, 3])
        else:
            print(5)
    except:
        continue

nan
nan
5
nan
newjersey
newjersey
nan
nan
nan
il
il
ma
ma
nan
nan
nan
indiana
indiana
bc
bc
ca
ca
ca
ca
edmonton
5
londdon
florida
florida
ca
ca
or
or
ca
ca
houston
5
ca
ca
ca
ca
iowa
iowa
florida
florida
ca
ca
ca
ca
ca
ca
florida
florida
rhodeisland
rhodeisland
ottawa
5
ca
ca
vancouver
5
ca
ca
md
md
youngstown
mn
mn
baldwinsville
5
ca
ca
ca
ca
ca
ca
louis
louis
ca
ca
ca
ca
ca
ca
ca
ca
ca
ca
princegeorge
5
ca
ca
indiana
indiana
ca
ca
louis
louis
setbal
ca
ca
ca
ca
roma
mn
mn
engen
northcarolina
northcarolina
chilliwack
5
helsinki
melbourne
5
tn
tn
ca
ca
ca
ca
ca
ca
arizona
arizona
london
ca
ca
ca
ca
niedernhausen
florida
florida
riga
or
or
peterborough
5
hamilton
5
newyork
5
houston
5
tn
tn
basel
ca
ca
ab
ab
ettlingen
ca
ca
columbus
olympia
tecumseh
or
or
calgary
5
dedensen
dedensen
ca
ca
york
ca
ca
ca
ca
carbondale
5
ca
ca
ca
ca
rhodeisland
rhodeisland
luxembourg
luxembourg
ca
ca
montreal
5
niederkassel
dortmund
ca
ca
badnauheim
essen
oklahomaciy
5
ca
ca
nedlands
coventry
ca
ca
il
il


In [139]:
users.isna().sum()

user_id                 0
location                0
age                 27830
location_city         187
location_state        596
location_country      195
dtype: int64

In [142]:
modify_location = users[(users['location_country'].isna())&(users['location_city'].notnull())]['location_city'].values

location_list = []
for location in modify_location:
    try:
        right_location = users[(users['location'].str.contains(location))&(users['location_country'].notnull())]['location'].value_counts().index[0]
        location_list.append(right_location)
    except:
        pass
for location in location_list:
    users.loc[users[users['location_city']==location.split(',')[0]].index,'location_state'] = location.split(',')[1]
    users.loc[users[users['location_city']==location.split(',')[0]].index,'location_country'] = location.split(',')[2]
    
    
modify_location = users[(users['location_country'].isna())&(users['location_state'].notnull())]['location_state'].values

location_list = []
for location in modify_location:
    try:
        right_location = users[(users['location'].str.contains(location))&(users['location_country'].notnull())]['location'].value_counts().index[0]
        location_list.append(right_location)
    except:
        pass
for location in location_list:
    users.loc[users[users['location_state']==location.split(',')[1]].index,'location_country'] = location.split(',')[2]
    
    
modify_location = users[(users['location_state'].isna())&(users['location_city'].notnull())&(users['location_country'].notnull())]['location_city'].values

location_list = []
for location in modify_location:
    try:
        right_location = users[(users['location'].str.contains(location))&(users['location_state'].notnull())]['location'].value_counts().index[0]
        location_list.append(right_location)
    except:
        pass
for location in location_list:
    users.loc[users[users['location_city']==location.split(',')[0]].index,'location_state'] = location.split(',')[1]

In [143]:
users.isna().sum()

user_id                 0
location                0
age                 27830
location_city         187
location_state        592
location_country      195
dtype: int64

In [144]:
users = users.dropna(subset=['location_city', 'location_state', 'location_country'], axis=0)

In [145]:
users.isna().sum()

user_id                 0
location                0
age                 27506
location_city           0
location_state          0
location_country        0
dtype: int64

In [146]:
users.to_csv('processed_users.csv')